In [36]:
import ssv
import pandas as pd

In [56]:
#####################################################
##### process input file in ssv format ##############
#####################################################

In [108]:
def read_ssv(filepath:str):
    raw_dat = ssv.loadf(filepath)[0]
    raw_dat_list = raw_dat[0].split('\n')[:-1] # data store in a list where each element represents a row: str
    col_names = raw_dat_list[1].split(' ') #get feature list
    
    data = [] #nested array
    for i in range(3,len(raw_dat_list)):
        data.append(raw_dat_list[i].rstrip().split(' ')) #rstrip remove trailing space from string
    
    df = pd.DataFrame.from_records(data, columns = col_names) #convert to dataframe   
    for col_name in df.columns: #encode all categorical variables
        if(df[col_name].dtype == 'object'):
            df[col_name]= df[col_name].astype('category')
            df[col_name] = df[col_name].cat.codes
    
    num_features, num_data = len(col_names), len(raw_dat_list) - 3
    
    return df, col_names, num_features, num_data

In [109]:
filepath = 'dt_hw1/noisy10_train.ssv'
df_train, col_names, num_features, num_data = read_ssv(filepath)

In [53]:
df_train.head()

,poisonous,capshape,capsurface,capcolor,bruises?,odor,gillattachment,gillspacing,gillsize,gillcolor,...,stalksurfacebelowring,stalkcolorabovering,stalkcolorbelowring,veiltype,veilcolor,ringnumber,ringtype,sporeprintcolor,population,habitat
0,0,5,0,2,1,5,1,0,0,7,...,2,2,5,0,0,1,3,1,5,0
1,0,0,3,6,1,0,1,0,0,0,...,2,5,5,0,0,1,3,1,3,3
2,1,2,3,7,0,2,1,0,0,4,...,1,0,4,0,0,1,1,0,5,1
3,1,2,2,0,1,2,1,0,0,4,...,2,5,5,0,0,1,3,0,4,1
4,0,2,3,2,1,5,1,0,0,6,...,2,4,5,0,0,1,3,1,4,0


In [110]:
col_names

['poisonous',
 'capshape',
 'capsurface',
 'capcolor',
 'bruises?',
 'odor',
 'gillattachment',
 'gillspacing',
 'gillsize',
 'gillcolor',
 'stalkshape',
 'stalkroot',
 'stalksurfaceabovering',
 'stalksurfacebelowring',
 'stalkcolorabovering',
 'stalkcolorbelowring',
 'veiltype',
 'veilcolor',
 'ringnumber',
 'ringtype',
 'sporeprintcolor',
 'population',
 'habitat']

In [54]:
print(num_features, ',', num_data)

23 , 2257


In [55]:
#########################################################
################## Entropy and info gain ##############################
#########################################################

In [61]:
import math

In [69]:
def entropy_binary(pos: int, neg: int):
    p0,p1 = neg/(pos+neg), pos/(pos+neg)
    if p0 == 0:
        bit0 = 0
    elif p1 == 0: bit1 == 0
    else:
        bit0, bit1 = math.log2(p0), math.log2(p1)
        
    return (- p0*bit0 - p1*bit1)
    

In [71]:
entropy_binary(80,20)

0.7219280948873623

In [ ]:
def data_entropy(data):
    

In [83]:
by_xy = df_train.groupby(['capshape','poisonous']).size().reset_index()
by_xy.columns = ['capshape','poisonous'] + ['xy_cnt']
by_xy

,capshape,poisonous,xy_cnt
0,0,0,108
1,0,1,21
2,1,1,2
3,2,0,577
4,2,1,400
5,3,0,6
6,3,1,7
7,4,0,9
8,4,1,2
9,5,0,659


In [88]:
by_x = df_train.groupby(['capshape']).size().reset_index()
by_x.columns = ['capshape'] + ['x_cnt']
row_total = by_x['x_cnt'].sum()
by_x['x_prob'] = by_x['x_cnt'] / row_total
by_x

,capshape,x_cnt,x_prob
0,0,129,0.057156
1,1,2,0.000886
2,2,977,0.432875
3,3,13,0.005760
4,4,11,0.004874
5,5,1125,0.498449


In [98]:
given_xy = by_xy.merge(by_x, on = 'capshape', how = 'inner')
given_xy['cond_prob'] = given_xy['xy_cnt']/given_xy['x_cnt']
given_xy['log2_cond_prob'] = given_xy['cond_prob'].apply(lambda x: math.log2(x))
given_xy['xy_entropy'] = given_xy['cond_prob'] * given_xy['log2_cond_prob']
given_xy

,capshape,poisonous,xy_cnt,x_cnt,x_prob,cond_prob,log2_cond_prob,xy_entropy
0,0,0,108,129,0.057156,0.837209,-0.256340,-0.214610
1,0,1,21,129,0.057156,0.162791,-2.618910,-0.426334
2,1,1,2,2,0.000886,1.000000,0.000000,0.000000
3,2,0,577,977,0.432875,0.590583,-0.759787,-0.448718
4,2,1,400,977,0.432875,0.409417,-1.288359,-0.527475
5,3,0,6,13,0.005760,0.461538,-1.115477,-0.514836
6,3,1,7,13,0.005760,0.538462,-0.893085,-0.480892
7,4,0,9,11,0.004874,0.818182,-0.289507,-0.236869
8,4,1,2,11,0.004874,0.181818,-2.459432,-0.447169
9,5,0,659,1125,0.498449,0.585778,-0.771575,-0.451971


In [103]:
cond_xy = given_xy[['capshape', 'x_prob', 'xy_entropy']].groupby('capshape').agg({'x_prob': 'mean', 'xy_entropy': 'sum'}).reset_index()

In [104]:
cond_xy['cond_xy_entropy'] = - cond_xy['xy_entropy'] * cond_xy['x_prob']
cond_xy['cond_xy_entropy'].sum()

0.95608719964100686

In [106]:
def partial_entropy_discrete(data, colname: str, target: str):
    #by_xy calculates the proportion by "colname" and "target" - both can take multiple discrete values  
    xy_cols = [colname, target]
    by_xy = df_train.groupby(xy_cols).size().reset_index()
    by_xy.columns = xy_cols + ['xy_cnt']
    
    #by_x calculates the proportion by "colname" - one of the features which takes multiple discrete values  
    by_x = df_train.groupby(colname).size().reset_index()
    by_x.columns = [colname] + ['x_cnt']
    row_total = by_x['x_cnt'].sum()
    by_x['x_prob'] = by_x['x_cnt'] / row_total
    
    #data to calculate conditional entropy H(Y|X = v) of Y given X = v (X is known to be equal to value "v")
    given_xy = by_xy.merge(by_x, on = colname, how = 'inner')
    given_xy['cond_prob'] = given_xy['xy_cnt']/given_xy['x_cnt']
    given_xy['log2_cond_prob'] = given_xy['cond_prob'].apply(lambda x: math.log2(x))
    given_xy['xy_entropy'] = given_xy['cond_prob'] * given_xy['log2_cond_prob']
    
    #data to calculate conditional entropy H(Y|X) of Y given X (X as a random variable)
    cond_xy = given_xy[[colname, 'x_prob', 'xy_entropy']].groupby(colname).agg({'x_prob': 'mean', 'xy_entropy': 'sum'}).reset_index()
    cond_xy['cond_xy_entropy'] = - cond_xy['xy_entropy'] * cond_xy['x_prob']
    
    #condition entropy H(Y|X)
    return cond_xy['cond_xy_entropy'].sum()
    

In [111]:
target, feat_col = col_names[0], col_names[1:]

In [113]:
for f in feat_col:
    print(f, ": ", partial_entropy_discrete(df_train, f, target))

capshape :  0.956087199641
capsurface :  0.967782087891
capcolor :  0.854093405505
bruises? :  0.874673138015
odor :  0.488631472725
gillattachment :  0.968772898231
gillspacing :  0.938432632202
gillsize :  0.943901462981
gillcolor :  0.844924795238
stalkshape :  0.78465828489
stalkroot :  0.913883313777
stalksurfaceabovering :  0.708375246734
stalksurfacebelowring :  0.718999419312
stalkcolorabovering :  0.764458148764
stalkcolorbelowring :  0.783385736406
veiltype :  0.96969294258
veilcolor :  0.966742025985
ringnumber :  0.963431212008
ringtype :  0.694766418217
sporeprintcolor :  0.632703550856
population :  0.909030733377
habitat :  0.913080667614


In [118]:
temp = df_train.groupby(target).size().reset_index()

In [128]:
temp.loc[temp['poisonous'] == 0][0].values[0]

1359

In [129]:
temp.loc[temp['poisonous'] == 1][0].values[0]

898

In [202]:
def MaxGainAttribute(data, target: str):

    temp = data.groupby(target).size().reset_index()
    pos, neg = temp.loc[temp[target] == 1][0].values[0], temp.loc[temp[target] == 0][0].values[0]
    
    entropy_orig = entropy_binary(pos, neg)
    
    max_info_gain = 0
    best_attr = ''
    idx = 0
    
    col = [l for l in list(data.columns) if l != target]
    for i, c in enumerate(col):
        info_gain = entropy_orig - partial_entropy_discrete(df_train, c, target)
        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_attr = c
            idx = i
    
    return best_attr

In [203]:
MaxGainAttribute(df_train, target)

'odor'

In [154]:
###############################################
#######decision tree###########################
###############################################

In [227]:
class Node:
    parent_node = None
    parent_value = None
    node_name = None
    attr_list = None
    def _init_(self, parent_node:str, node_value:int, node_name:str, attr_list:list):
        self.parent_node = parent_node #parent node
        self.node_value = node_value #current node value in discrete
        self.attr_list = attr_list #current available attributes to be put into children node
        self.node_name = node_name #current node name

In [228]:
class DecisionTree:
    children = []
    
    def _init_(self, data, target:str, node):
        self.data = data
        self.target = target
        self.node = Node()

        #self.node.node_name = MaxGainAttribute(data, target)
    
    def grow_tree(self, data, target:str, node = Node()):        
        #pick the best performed attributs and put it into current examing node
        node.node_name = MaxGainAttribute(data, target)
        print(node.node_name)
        node.value = data[node.node_name].unique()
        
        temp = data.drop(node.node_name, axis = 1)
        for v in node.value:
            idx = data.index[data[node.node_name] == v].tolist()
            child_data = temp.iloc[idx,:]
            node.node_name = MaxGainAttribute(data, target)
            
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        